In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from passivbot_isolated_margin import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter
from common_functions import sort_dict_keys

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
user = 'your_user_name'
settings = load_settings(user)
settings

In [ ]:
try:
    await bot.cc.close()
except:
    pass
bot = await create_bot(settings)

In [ ]:
ss = sorted(bot.balance)
bdf = pd.DataFrame(
    {'equity': [bot.balance[s]['equity'] for s in ss],
     'debt': [bot.balance[s]['debt'] for s in ss],
     'onhand': [bot.balance[s]['onhand'] for s in ss],
     'total_account_equity': [bot.balance[s]['total_account_equity'] for s in ss],
     'total_account_debt': [bot.balance[s]['total_account_debt'] for s in ss],
     'coin_equity': [bot.balance[s][bot.s2c[s]]['equity'] for s in ss],
     'quot_equity': [bot.balance[s]['BTC']['equity'] for s in ss],
     'coin_onhand': [bot.balance[s][bot.s2c[s]]['onhand'] for s in ss],
     'quot_onhand': [bot.balance[s]['BTC']['onhand'] for s in ss],
     'coin_borrowable': [bot.balance[s][bot.s2c[s]]['borrowable'] for s in ss],
     'quot_borrowable': [bot.balance[s]['BTC']['borrowable'] for s in ss],
    }, index=ss)
bdf.sort_values('equity')

In [ ]:
ss = bot.symbols
analyses = {}
n_days_ = 40
mts = {}
age_limit = (time() - 60 * 60 * 24 * n_days_) * 1000
for s in ss:
    mtdf = pd.DataFrame(bot.my_trades[s]).set_index('timestamp').sort_values('id')
    mtdf = mtdf[mtdf.index > age_limit]
    mts[s] = mtdf
    threshold = bot.my_trades_analysis[s]['entry_exit_amount_threshold']
    buys = mtdf[mtdf.side == 'buy']
    sels = mtdf[mtdf.side == 'sell']
    le = buys[buys.amount < threshold]
    se = sels[sels.amount < threshold]
    lx = sels[sels.amount >= threshold]
    sx = buys[buys.amount >= threshold]
    bvwap = buys.cost.sum() / bas if (bas := buys.amount.sum()) > 0 else 0
    svwap = sels.cost.sum() / sas if (sas := sels.amount.sum()) > 0 else 0
    rvwap = svwap / bvwap if bvwap and svwap else 1.0
    rlvwap = (lx.cost.sum() / lx.amount.sum()) / (le.cost.sum() / le.amount.sum())
    rsvwap = (se.cost.sum() / se.amount.sum()) / (sx.cost.sum() / sx.amount.sum())
    

    
    
    analyses[s] = {'rvwap': round(rvwap, 6), 'n_trades': len(mtdf),
                   'vol': round(mtdf.cost.sum(), 6), 'rlvwap': round(rlvwap, 6),
                   'rsvwap': rsvwap, 'bvol': round(buys.cost.sum(), 6), 'svol': round(sels.cost.sum(), 6),
                   'n_days': (mtdf.index[-1] - mtdf.index[0]) / DAY_TO_MILLIS}
    anl = bot.my_trades_analysis[s]
    nanl = {
        'long_bag_duration_days': (bot.cc.milliseconds() - anl['long_start_ts']) / DAY_TO_MILLIS,
        'shrt_bag_duration_days': (bot.cc.milliseconds() - anl['shrt_start_ts']) / DAY_TO_MILLIS,
        'long_cost': anl['long_cost'],
        'shrt_cost': anl['shrt_cost'],
        'long_vwap_last_price_diff': anl['long_vwap'] / bot.last_price[s],
        'shrt_vwap_last_price_diff': bot.last_price[s] / anl['shrt_vwap'] if anl['shrt_vwap'] else 1.0,
        'max_min_range': mtdf.price.max() / mtdf.price.min(),
        'entry_delay_minutes': bot.balance[s]['entry_cost'] / (bot.settings[s]['account_equity_pct_per_hour'] *
                                                               bot.balance[s]['total_account_equity']) * 60,
        'avg_n_entries_per_day': (len(le) + len(se)) / analyses[s]['n_days'],
    }
    nanl['long_cost_times_price_diff'] = nanl['long_cost'] * nanl['long_vwap_last_price_diff']
    nanl['shrt_cost_times_price_diff'] = nanl['shrt_cost'] * nanl['shrt_vwap_last_price_diff']
    analyses[s] = {**analyses[s], **nanl}
anldf = pd.DataFrame(analyses).T
allmt = pd.concat(mts.values(), axis=0).sort_index()
anldf.sort_values('vol')

In [ ]:
s = 'XMR/BTC'
await bot.update_my_trades(s)
mtdf = pd.DataFrame(bot.my_trades[s]).set_index('timestamp').sort_values('id')
mtdf = mtdf[mtdf.index > age_limit]
threshold = bot.my_trades_analysis[s]['entry_exit_amount_threshold']
buys = mtdf[mtdf.side == 'buy']
sels = mtdf[mtdf.side == 'sell']
le = buys[buys.amount < threshold]
se = sels[sels.amount < threshold]
lx = sels[sels.amount >= threshold]
sx = buys[buys.amount >= threshold]

In [ ]:
# last n trades
mtdf.tail(30)

In [ ]:
print(s)
bot.get_ideal_orders(s)
pp.pprint(bot.ideal_orders[s])
c, q = bot.symbol_split[s]
anl = bot.my_trades_analysis[s]
bal = bot.balance[s]
le.price.plot(style='b.--')
se.price.plot(style='r.--')
lx.price.plot(style='ro')
sx.price.plot(style='bo')
if bot.ideal_orders[s]['long_sel']['price']:
    plt.plot([mtdf.index[-1]], [bot.ideal_orders[s]['long_sel']['price']], 'rx')
if bot.ideal_orders[s]['shrt_buy']['price']:
    plt.plot([mtdf.index[-1]], [bot.ideal_orders[s]['shrt_buy']['price']], 'bx')
anl